## PCA를 통해 차원 축소한 데이터를 이용한 모델링
   1. 각 Ingredient를 column으로 변경한 뒤 PCA를 통해 차원을 축소한다.
   2. tfidfVectorizer를 이용하여 학습을 했을 때, f1 score가 가장 높았던 Xgboost와 Neural-network 이용하여 학습한다.

In [1]:
import pandas as pd
import numpy as np

% matplotlib inline
import matplotlib.pyplot as plt

from collections import Counter
from pprint import pprint

pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)

import re
from nltk.stem import WordNetLemmatizer

# Xgboost module
import xgboost as xgb
from sklearn.neural_network import MLPClassifier

# estimate를 위한 modules
from sklearn.model_selection import cross_val_score
from sklearn import cross_validation
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

# featrures를 encoding 하기 위한 modules
from sklearn.preprocessing import LabelEncoder
from gensim.models import word2vec

# Data set을 나누기 위한 modules
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

c:\Python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\Python36\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\Python36\lib\site-packages\scipy\sparse\sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## I. DATA LOAD

In [2]:
train = pd.read_json('train.json', encoding = 'UTF-8')

COPY

In [3]:
df = train.copy()

print(df.shape)
df.tail(2)

(39774, 3)


,cuisine,id,ingredients
39772,chinese,41882,"[boneless chicken skinless thigh, minced garli..."
39773,mexican,2362,"[green chile, jalapeno chilies, onions, ground..."


## II. Pre-processing

1. 대문자를 소문자로
2. 상품명 제거 : Oscar Mayer Deli Fresh Smoked,  '®, ™'가 있는 것도 상품명
3. 재료손질 & 재료상태 제거 : chopped, ground, fresh, powdered, sharp, crushed, grilled, roasted, sliced, cooked, shredded, cracked, minced, finely...
4. ','(콤마) 뒤에 있는 것은 재료 손질 방법이므로 불필요하다고 생각됨
5. digit(숫자) 제거
6. 특수문자 제거 : %, -, (, ), '.', oz. , ....
7. lemmatize를 이용하여 단어를 원형으로 변경

In [4]:
### 각 row 마다의 recipe 별 ingredient를 count하기 위한 작업

# 여기서는 ingredient가 각 1개씩 count 될 것이다.
bag_of_ingredients = [Counter(ingredient) for ingredient in df.ingredients]

# 각 ingredients의 종류별 개수
sum_of_ingredients = sum(bag_of_ingredients, Counter())

########################################################################################

### sum_of_ingredients를 dataframe에 넣기 위한 작업

# dict -> list -> dataframe
sum_of_ingredients_dict = dict(sum_of_ingredients)
sum_of_ingredients_list = list(sum_of_ingredients_dict.items())

ingredients_df = pd.DataFrame(sum_of_ingredients_list)
ingredients_df.columns = ['ingredient', 'count']
ingredients_df.tail(2)

print('전처리 전 ingredient는 총 {}개 입니다.'.format(len(ingredients_df)))

전처리 전 ingredient는 총 6714개 입니다.


In [5]:
def pre_processing_(recipe):
    
    wnl = WordNetLemmatizer()
    
    # 1. lower 함수를 이용하여 대문자를 소문자로 변경
    recipe = [str.lower(ingredient) for ingredient in recipe]
    
    # 2. 상품명을 제거하는 함수
    def delete_brand_(ingredient):
        
        # '®'이 있는 브랜드
        ingredient = re.sub("country crock|i can't believe it's not butter!|bertolli|oreo|hellmann's"
                            , '', ingredient)
        ingredient = re.sub("red gold|hidden valley|original ranch|frank's|redhot|lipton", '', ingredient)
        ingredient = re.sub("recipe secrets|eggland's best|hidden valley|best foods|knorr|land o lakes"
                            , '', ingredient)
        ingredient = re.sub("sargento|johnsonville|breyers|diamond crystal|taco bell|bacardi", '', ingredient)
        ingredient = re.sub("mccormick|crystal farms|yoplait|mazola|new york style panetini", '', ingredient)
        ingredient = re.sub("ragu|soy vay|tabasco|truvía|crescent recipe creations|spice islands", '', ingredient)
        ingredient = re.sub("wish-bone|honeysuckle white|pasta sides|fiesta sides", '', ingredient)
        ingredient = re.sub("veri veri teriyaki|artisan blends|home originals|greek yogurt|original ranch"
                            , '', ingredient)
        ingredient = re.sub("jonshonville", '', ingredient)

        # '™'이 있는 브랜드
        ingredient = re.sub("old el paso|pillsbury|progresso|betty crocker|green giant|hellmannâ€", '', ingredient)

        # 'oscar mayer deli fresh smoked' 브랜드
        ingredient = re.sub("oscar mayer deli fresh smoked", '', ingredient)
        
        return ingredient
    
    # 3. 재료 손질, 상태를 제거하는 함수
    def delete_state_(ingredient):
    
        ingredient = re.sub('frozen|chopped|ground|fresh|powdered', '', ingredient)
        ingredient = re.sub('sharp|crushed|grilled|roasted|sliced', '', ingredient)
        ingredient = re.sub('cooked|shredded|cracked|minced|finely', '', ingredient)        
     
        return ingredient
    
    # 4. 콤마 뒤에 있는 재료손질방법을 제거하는 함수
    def delete_comma_(ingredient):
        
        ingredient = ingredient.split(',')
        ingredient = ingredient[0]
        
        return ingredient

    ## 그외 전처리 함수 (숫자제거, 특수문자제거, 원형으로변경)
    def original_(ingredient):
        
        # 숫자제거
        ingredient = re.sub('[0-9]', '', ingredient)
        
        # 특수문자 제거
        ingredient = ingredient.replace("oz.", '')
        ingredient = re.sub('[&%()®™/]', '', ingredient)
        ingredient = re.sub('[-.]', '', ingredient)
        
        # lemmatize를 이용하여 단어를 원형으로 변경
        ingredient = wnl.lemmatize(ingredient)
        
        return ingredient

    # 양 끝 공백을 제거하는 함수
    def delete_space_(ingredient):
        
        ingredient = ingredient.strip()
        
        return ingredient

    recipe = [delete_brand_(ingredient) for ingredient in recipe]
    recipe = [delete_state_(ingredient) for ingredient in recipe]
    recipe = [delete_comma_(ingredient) for ingredient in recipe]
    recipe = [original_(ingredient) for ingredient in recipe]
    recipe = [delete_space_(ingredient) for ingredient in recipe]

    return recipe

In [6]:
df['ingredients'] = df['ingredients'].apply(lambda x : pre_processing_(x))

#### Pre-processing 후 Data set의 변화를 보고, 알맞게 전처리가 이루어 졌는지 확인한다.

In [7]:
%%time
### 각 row 마다의 recipe 별 ingredient를 count하기 위한 작업

# 여기서는 ingredient가 각 1개씩 count 될 것이다.
bag_of_ingredients = [Counter(ingredient) for ingredient in df.ingredients]

# 각 ingredients의 종류별 개수
sum_of_ingredients = sum(bag_of_ingredients, Counter())

########################################################################################

### sum_of_ingredients를 dataframe에 넣기 위한 작업

# dict -> list -> dataframe
sum_of_ingredients_dict = dict(sum_of_ingredients)
sum_of_ingredients_list = list(sum_of_ingredients_dict.items())

ingredients_df = pd.DataFrame(sum_of_ingredients_list)
ingredients_df.columns = ['ingredient', 'count']
ingredients_df.tail(2)

print('전처리 후 ingredient는 총 {}개 입니다.'.format(len(ingredients_df)))

전처리 후 ingredient는 총 6313개 입니다.
Wall time: 4min 48s


## III. recipe 안에 있는 ingredients를 column으로 만들기

In [8]:
# 1. all_ingredients set에 ingredients들을 담는다.
all_ingredients = set()
df['ingredients'].apply(lambda x : [all_ingredients.add(i) for i in list(x)])
#print(all_ingredients)

# 'ingredient' columns를 새로 만들면서, 각 ingredient가 해당 row의 recipe에 들어 있으면 True, 그렇지 않으면 False를 반환하게 함
for ingredient in all_ingredients:
    df[ingredient] = df['ingredients'].apply(lambda x : ingredient in x)

In [9]:
len(df.columns)

6316

In [10]:
df.head(1) #cuisine, id, ingredients

cuisine     id                                        ingredients         \
0   greek  10259  [romaine lettuce, black olives, grape tomatoes...  False   

   spring! water  acai juice  jamaican jerk rub  flavored tortilla chips  \
0          False       False              False                    False   

   reduced sodium beef stock  yakinori  buttermilk  risotto  \
0                      False     False       False    False   

   vanilla lowfat  yogurt  mexican oregano  orange bitters  \
0                   False            False           False   

   candied citron peel  linguine  tartlet shells  preserv raspberri seedless  \
0                False     False           False                       False   

   almond  vegetable oil cooking spray  fat free yogurt  bass fillets  \
0   False                        False            False         False   

   mexican seasoning mix  navy beans  pie pastry  duckling  gluten flour  \
0                  False       False       False     False         False   

   center cut bacon  cornichons  pita bread rounds  lobster  \
0             False       False              False    False   

   san marzano tomatoes  provolone cheese  mandarin orange segments  \
0                 False             False                     False   

   digestive biscuit   bean  sea bream  dried orange peel  himalayan salt  \
0              False  False      False              False           False   

   ginseng   carp  lambs liver  chuck roast  light molasses  calabash  \
0    False  False        False        False           False     False   

   bread crumb  fat free cream of mushroom soup  rustic bread  \
0        False                            False         False   

   chocolate covered english toffee  jerk paste  ancho chili  pepper  \
0                             False       False                False   

   carrot juice  mint sauce  zucchini  seafood stock  black rice  \
0         False       False     False          False       False   

   nonfat evaporated milk  boneless duck breast halves  white pepper  fatback  \
0                   False                        False         False    False   

   custard  cream of potato soup  chocolate cookie crumbs  glutenfree pasta  \
0    False                 False                    False             False   

   fatfree buttermilk  lowfat flour tortillas    cod  pepperoni  \
0               False                   False  False      False   

   reduced fat mexican cheese   drum  regular or convert rice  \
0                       False  False                    False   

   crystallized ginger  flax seeds  low sodium beef broth  all purpose sticks  \
0                False       False                  False               False   

   arrabbiata sauce  chinese parsley  small curd cottage cheese  \
0             False            False                      False   

   multigrain penne pasta  low sodium low fat pasta sauce  large shrimp  \
0                   False                           False         False   

   low fat coleslaw dressing  mashed banana  white baking bar  \
0                      False          False             False   

   extra lean  beef  garlic salt  bigoli  mixed dried fruit  whole grain buns  \
0             False        False   False              False             False   

   mackerel fillets  branzino  nacho cheese tortilla chips  beef liver  \
0             False     False                        False       False   

   dashi powder  romaine lettuce hearts  mexican chocolate  reblochon  \
0         False                   False              False      False   

   kumquats in syrup  trout fillet    rib  orange roughy    ahi  \
0              False         False  False          False  False   

   dri thyme leaves  enokitake  amaretto liqueur  dry hard cider  \
0             False      False             False           False   

   unsulphured molasses  vodka sauc made with  cream  smoked mackerel  pisco  \
0              

In [12]:
column_list = []
for col in df.columns:
    column_list.append(col)

print(type(column_list), len(column_list))

<class 'list'> 6316


In [13]:
column_list.remove('id')
column_list.remove('ingredients')
column_list.remove('cuisine')

In [14]:
len(column_list)
print(column_list[:10])

['', 'spring! water', 'acai juice', 'jamaican jerk rub', 'flavored tortilla chips', 'reduced sodium beef stock', 'yakinori', 'buttermilk', 'risotto', 'vanilla lowfat  yogurt']


In [15]:
%%time
df[column_list] = df[column_list].astype(int) # False는 0으로, True는 1로

Wall time: 9min 56s


COPY

In [16]:
df_dummy = df.copy()

In [17]:
# 'id'와 'ingredients' columns는 더이상 필요가 없으므로 지운다.
del df_dummy['id']
del df_dummy['ingredients']

In [18]:
# 'cuisine' column을 지우기 위해 df_dummy를 copy
df_features = df_dummy.copy()

del df_features['cuisine']
df_features.tail(1)

spring! water  acai juice  jamaican jerk rub  \
39773  0              0           0                  0   

       flavored tortilla chips  reduced sodium beef stock  yakinori  \
39773                        0                          0         0   

       buttermilk  risotto  vanilla lowfat  yogurt  mexican oregano  \
39773           0        0                       0                0   

       orange bitters  candied citron peel  linguine  tartlet shells  \
39773               0                    0         0               0   

       preserv raspberri seedless  almond  vegetable oil cooking spray  \
39773                           0       0                            0   

       fat free yogurt  bass fillets  mexican seasoning mix  navy beans  \
39773                0             0                      0           0   

       pie pastry  duckling  gluten flour  center cut bacon  cornichons  \
39773           0         0             0                 0           0   

       pita bread rounds  lobster  san marzano tomatoes  provolone cheese  \
39773                  0        0                     0                 0   

       mandarin orange segments  digestive biscuit  bean  sea bream  \
39773                         0                  0     0          0   

       dried orange peel  himalayan salt  ginseng  carp  lambs liver  \
39773                  0               0        0     0            0   

       chuck roast  light molasses  calabash  bread crumb  \
39773            0               0         0            0   

       fat free cream of mushroom soup  rustic bread  \
39773                                0             0   

       chocolate covered english toffee  jerk paste  ancho chili  pepper  \
39773                                 0           0                    0   

       carrot juice  mint sauce  zucchini  seafood stock  black rice  \
39773             0           0         0              0           0   

       nonfat evaporated milk  boneless duck breast halves  white pepper  \
39773                       0                            0             0   

       fatback  custard  cream of potato soup  chocolate cookie crumbs  \
39773        0        0                     0                        0   

       glutenfree pasta  fatfree buttermilk  lowfat flour tortillas  cod  \
39773                 0                   0                       0    0   

       pepperoni  reduced fat mexican cheese  drum  regular or convert rice  \
39773          0                           0     0                        0   

       crystallized ginger  flax seeds  low sodium beef broth  \
39773                    0           0                      0   

       all purpose sticks  arrabbiata sauce  chinese parsley  \
39773                   0                 0                0   

       small curd cottage cheese  multigrain penne pasta  \
39773                          0                       0   

       low sodium low fat pasta sauce  large shrimp  \
39773                               0             0   

       low fat coleslaw dressing  mashed banana  white baking bar  \
39773                          0              0                 0   

       extra lean  beef  garlic salt  bigoli  mixed dried fruit  \
39773                 0            0       0                  0   

       whole grain buns  mackerel fillets  branzino  \
39773                 0                 0         0   

       nacho cheese tortilla chips  beef liver  dashi powder  \
39773                            0           0             0   

       romaine lettuce hearts  mexican chocolate  reblochon  \
39773                       0                  0          0   

       kumquats in syrup  trout fillet  rib  orange roughy  ahi  \
39773                  0             0    0              0    0   

       dri thyme leaves  enokitake  amaretto liqueur  dry hard cider  \
39773                 0          0                 0               0   

       

In [19]:
len(df_features.columns)

6313

### Label encoding

In [20]:
Lec = LabelEncoder()
train_target_value = Lec.fit_transform(df_dummy['cuisine'].values)

print(train_target_value.shape)

(39774,)


In [21]:
print(train_target_value[:10])

[ 6 16  4  7  7 10 17  9 13  9]


In [22]:
print(Lec.classes_)

['brazilian' 'british' 'cajun_creole' 'chinese' 'filipino' 'french' 'greek'
 'indian' 'irish' 'italian' 'jamaican' 'japanese' 'korean' 'mexican'
 'moroccan' 'russian' 'southern_us' 'spanish' 'thai' 'vietnamese']


### Data Split

In [23]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(df_features, train_target_value)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(29830, 6313) (9944, 6313) (29830,) (9944,)


## IV. PCA

In [24]:
%%time
# fit method를 호출하여 주성분을 찾는다. 주성분은 200개로 한다.
pca = PCA(n_components=200, whiten=True, random_state=0).fit(X_train)
# transform method를 호출해 데이터를 회전시키고 차원을 축소
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

print("X_train_pca.shape: {}".format(X_train_pca.shape))

X_train_pca.shape: (29830, 200)
Wall time: 49.1 s


In [25]:
pca.components_

array([[  3.01603240e-03,   1.13650490e-04,  -6.19415086e-05, ...,
         -1.74279935e-04,  -4.76798484e-04,   1.18977788e-05],
       [ -2.96578422e-03,   1.21930493e-06,  -3.71489446e-05, ...,
          1.17364157e-04,   4.39622348e-04,  -8.35092568e-05],
       [ -1.01337096e-03,   7.10053443e-05,   4.80955014e-05, ...,
         -7.26010761e-05,   9.56424132e-05,   1.01664281e-04],
       ..., 
       [  2.45932021e-03,  -9.43243879e-06,   2.08686535e-04, ...,
          5.22421706e-04,  -1.11903169e-03,   3.03246891e-04],
       [ -3.99060610e-02,  -2.48516480e-04,  -8.78813425e-04, ...,
         -2.29300827e-03,  -2.26765919e-03,  -1.70007468e-04],
       [ -4.57679291e-02,  -5.16452072e-05,  -1.78381778e-05, ...,
         -1.52948679e-03,   1.16166043e-03,   1.56817469e-04]])

이제 새로운 data는 처음 200개의 주성분에 해당하는 특성을 갖는다.

## V. Model Selection

### 1. Xgboost

In [34]:
def Xgboost_():
    xgbr = xgb.XGBClassifier(
        n_estimators = 400,
        max_depth = 5
    ).fit(X_train_pca, y_train)
        
    cuisine = ['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino', 'french', 'greek',
               'indian', 'irish', 'italian', 'jamaican', 'japanese', 'korean', 'mexican', 'moroccan',
               'russian', 'southern_us', 'spanish', 'thai', 'vietnamese']

    print (classification_report(y_test, xgbr.predict(X_test_pca), digits=4, target_names=cuisine))
    return print("Test score : {}".format(xgbr.score(X_test_pca, y_test)))

In [35]:
%%time
Xgboost = Xgboost_()

              precision    recall  f1-score   support

   brazilian     0.7647    0.2149    0.3355       121
     british     0.4776    0.1624    0.2424       197
cajun_creole     0.7585    0.5899    0.6637       378
     chinese     0.6882    0.8021    0.7408       652
    filipino     0.7048    0.3854    0.4983       192
      french     0.4963    0.5069    0.5015       655
       greek     0.8110    0.4733    0.5978       281
      indian     0.8125    0.8486    0.8301       766
       irish     0.5890    0.2654    0.3660       162
     italian     0.6580    0.8759    0.7515      1885
    jamaican     0.8750    0.4532    0.5972       139
    japanese     0.7631    0.5706    0.6529       333
      korean     0.7561    0.5822    0.6578       213
     mexican     0.8150    0.8824    0.8474      1633
    moroccan     0.8378    0.5741    0.6813       216
     russian     0.7949    0.2605    0.3924       119
 southern_us     0.5652    0.7312    0.6376      1120
     spanish     0.5904    

### 2. Neural Network

In [32]:
def Neural_network_():
    nn = MLPClassifier(hidden_layer_sizes=(400,500,400))
    nn.fit(X_train_pca, y_train)
    
    cuisine = ['brazilian', 'british', 'cajun_creole', 'chinese', 'filipino', 'french', 'greek',
               'indian', 'irish', 'italian', 'jamaican', 'japanese', 'korean', 'mexican', 'moroccan',
               'russian', 'southern_us', 'spanish', 'thai', 'vietnamese']

    print (classification_report(y_test, nn.predict(X_test_pca), digits=4, target_names=cuisine))
    return print("Test score : {}".format(nn.score(X_test_pca, y_test)))

In [33]:
%%time
Neural_Network = Neural_network_()

              precision    recall  f1-score   support

   brazilian     0.4725    0.3554    0.4057       121
     british     0.2810    0.1726    0.2138       197
cajun_creole     0.5940    0.6270    0.6100       378
     chinese     0.7342    0.7117    0.7227       652
    filipino     0.5704    0.4219    0.4850       192
      french     0.4711    0.4855    0.4782       655
       greek     0.6667    0.5480    0.6016       281
      indian     0.8210    0.8081    0.8145       766
       irish     0.3618    0.3395    0.3503       162
     italian     0.7087    0.8143    0.7578      1885
    jamaican     0.7363    0.4820    0.5826       139
    japanese     0.6272    0.6366    0.6319       333
      korean     0.7299    0.5962    0.6563       213
     mexican     0.8291    0.8677    0.8480      1633
    moroccan     0.6198    0.6944    0.6550       216
     russian     0.5208    0.2101    0.2994       119
 southern_us     0.5966    0.6509    0.6225      1120
     spanish     0.4229    

## VI. Conclusion

- 성능을 더 높이고 싶어서 시도해보았지만, PCA를 이용하여 축소된 데이터를 가지고 학습시킨 경우 더 성능이 좋지 않았다.
- 각 ingredient의 연관성과 가중치로 encoding한 tfidfVetorizer가 학습에 더 좋은 효과를 주는 것이라 판단했다.